In [12]:
from bs4 import BeautifulSoup, NavigableString
from urllib import request, parse
import re, time

In [13]:
BIO_LINKS = set()
fon = 'bio300to510AD_Century.txt'
fo  = open(fon, "w", encoding="utf-8", newline="\n")


In [14]:
base_url = 'zh.wikipedia.org'

In [15]:
centuries = [(int(n/100)+1) for n in list(range(300, 511, 100))]
centuries

[4, 5, 6]

In [16]:
for century in centuries:
    url = fr"{base_url}/wiki/Category:{century}世紀出生"
    print(url)
    req  = request.Request('http://'+parse.quote(url), headers={'User-Agent': 'Mozilla/5.0'})
    try:
        page = request.urlopen(req)
    except:
        continue
    soup = BeautifulSoup(page, 'lxml')
    nodes = soup.find_all('a', {'title':re.compile(r'[一-鿕]+'), 'href':re.compile(r'/wiki/%')}, recursive=True, limit=2000)
    for child in nodes:
        link = child['href']
        if link.startswith('/wiki'):
            article = f"{base_url}{child['href']}"
            article = article.replace(r'/wiki/', r'/zh-tw/')
            fo.write(f"{article}\n")
            BIO_LINKS.add(article)
    
    time.sleep(0.5)

fo.close()

zh.wikipedia.org/wiki/Category:4世紀出生
zh.wikipedia.org/wiki/Category:5世紀出生
zh.wikipedia.org/wiki/Category:6世紀出生


In [51]:
fo.close()

In [17]:
fon = 'bio300to510AD_Century_initial_text.txt'
fo  = open(fon, "w", encoding="utf-8", newline="\n")

In [18]:
BeginTime = time.time()

cnt = 0
for link in BIO_LINKS:
    cnt += 1
    if cnt % 50 == 0:
        fo.flush()
        print(cnt)
    fo.write(link + "\n")
    req  = request.Request(f"http://{link}", headers={'User-Agent': 'Mozilla/5.0'})
    try:
        page = request.urlopen(req)
    except:
        continue
    soup = BeautifulSoup(page, 'lxml')
    headword = ''
    h1s = soup.find_all('h1')
    if len(h1s) > 0:
        headword = h1s[0].text
    fo.write(headword + "\n")
        
    nodes = soup.find_all('div', {'class':'mw-parser-output'})
    ## in each "div", the first <p></p> seems to contain the information (name, YOB, YOD, courtesy name, place names) 
    
    if len(nodes) > 0:
        #print(len(nodes))
        div = nodes[0]
        basic_info = list(div.find_all('p', recursive=False))
        #print(f"No. of <p> nodes: {len(basic_info)}")
        for paragraph in basic_info:  # <p></p>
            children = list(paragraph.children)
            if children[0].name == 'a': # if this <p> node contains a URL link, skip
                print(f"Skipping node {str(paragraph)}")
                continue
            else:
                if isinstance(paragraph, NavigableString):
                    fo.write(paragraph.strip() + "\n")
                else:
                    fo.write(paragraph.text.strip() + "\n")
                break # done!
        fo.write('-'*20 + "\n")
    else:
        fo.write("No info available for this entry\n")
    #if cnt > 50:
    #    break

fo.close()
EndTime = time.time()
ElapsedTime = EndTime - BeginTime
print("Done! Total time elapsed: [%.4f] sec(s)" % ElapsedTime) if True else ""

50
100
150
200
250
300
Done! Total time elapsed: [328.2433] sec(s)


In [47]:
fo.close()